In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import re
from cleantext import clean

In [2]:
# Keyword & date range
query = "russian inflation until:2023-05-05 since:2023-01-01"
tweets = []
# Maximum tweets to be scraped
limit = 10
# keyword check, because some tweet may be don't have the keyword that we want
check = "inflation"


for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if len(tweets) == limit:
        break
    else :
        content = tweet.content.split()
        if check in content:
            # remove emoji
            clean_tweet = clean(tweet.content, no_emoji=True)
            # remove line break
            clean_tweet = clean_tweet.replace("\n", " ")
            # remove hashtag
            clean_tweet = re.sub(r"#(\w+)", "", clean_tweet)
            # remove annoying data
            clean_tweet = re.sub(r"(?:\@|https?\://)\S+", "", clean_tweet)
            # remove whitespace
            clean_tweet = " ".join(clean_tweet.split())
            # just use the tweet and date
            tweets.append([tweet.date, clean_tweet])

df = pd.DataFrame(tweets, columns=['date', 'tweet'])
# cleaning the remaining dirty tweets
tweets_clean = df['tweet'].apply(lambda x: clean(x, fix_unicode=True, to_ascii=True, lower=True, no_line_breaks=True, no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True, no_punct=True, replace_with_url="", replace_with_email="", replace_with_phone_number="", replace_with_currency_symbol=""))

# combine date and tweet
tweet_fix = pd.concat([df['date'], tweets_clean], axis=1)

# remove duplicate tweets
tweet_fix = tweet_fix.drop_duplicates(subset=['tweet'], keep='first')

# save to csv
tweet_fix.to_csv('./data/russian_inflation.csv', index=False, header=['date', 'tweet'])

C:\Users\Asus\AppData\Local\Temp\ipykernel_18720\277455601.py:14: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  content = tweet.content.split()
C:\Users\Asus\AppData\Local\Temp\ipykernel_18720\277455601.py:17: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  clean_tweet = clean(tweet.content, no_emoji=True)
Stopping after 20 empty pages
